In [ ]:
# 在本机未更新至2.0.8的情况下，使用vnpy2.0.8的回测逻辑，2.0.8修复了一些2.0.7的回测bug
import sys
from pathlib import Path
new_version_path = Path(r'D:\vnpy-2.0.8')
sys.path.insert(0, str(new_version_path))
# sys.path

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from datetime import datetime
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting, ver_test
from vnpy.trader.database import database_manager
from vnpy.trader.utility import extract_vt_symbol

from double_ma_strategy import DoubleMaStrategy
from utility import vt_bar_to_df

In [ ]:
# 从数据库载入数据
symbol, exchange = extract_vt_symbol('RB888.SHFE')
data = database_manager.load_bar_data(
    symbol,
    exchange,
    '1h',
    start=datetime(2018, 1, 1),
    end=datetime(2019, 12, 1),
)

df = vt_bar_to_df(engine.history_data)
df.reset_index(inplace=True)
df.head()

In [ ]:
# 用matplot的交互式窗口查看数据是否正常
# 按住右键上下左右移动可以上下左右缩放，按住左键可以平移
%matplotlib notebook
df.close.plot(figsize=(12, 6))
plt.show()

In [ ]:
# 设置回测参数

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="RB888.SHFE",
    interval="1h",
    start=datetime(2018, 1, 1),
    end=datetime(2019, 12, 1),
    rate=1/10000,
    slippage=1,
    size=10,
    pricetick=1,
    capital=100000,
)
engine.add_strategy(DoubleMaStrategy, {})

In [ ]:
# 读取数据
engine.load_data()

df = vt_bar_to_df(engine.history_data)
df.reset_index(inplace=True)

df.head()

In [ ]:
# 进行连续合约的回测

In [ ]:
# 进行分段合约的回测